# Images inspection

In [2]:
import os
import pandas as pd
import SimpleITK as sitk
import tqdm
from metadata import *
import numpy as np
from numpy import random

# Check working directory:
cwd = os.getcwd()
print(cwd)

/home/alex/src


### Functions

In [1]:
score_selection = 1     # 1: MedAS, 2: CA

def process_dataset(path_dataset, score_csv_path = None):
    # Get the name of the dataset from path:
    dts_name = path_dataset.split('/')[-1]
    
    # Create an empty dictionary:
    df = {}
    
    # Create an empty dictionary with the datasets as keys:
    df[dts_name] = {}
    
    # We read all modalities in the dataset folder (T1, T1_bias_c, T2_FLAIR, etc.) 
    modalities = [f for f in os.listdir(path_dataset) if not f.startswith('.') and os.path.isdir(path_dataset+'/'+f)]
    
    # Create an empty key with each id in the folders of the dataset:
    for modality in modalities:
        # Create a list of all files in the T1 and T2_FLAIR directories avoiding hidden files:
        ID_list = [f for f in os.listdir(path_dataset + '/' + modality) if not f.startswith('.')]
        
        for id in ID_list:
            # Check if the ID is already in the dictionary. If not, add it with the dataset:
            if id not in df[dts_name].keys():
                df[dts_name][id] = {}

    # Read score file:
    if score_csv_path is not None:
        score_df = pd.read_csv(score_csv_path, sep=';')
    
    # Add their info:
    for id in tqdm.tqdm(df[dts_name].keys()):
        # Add score as a key for each id in the dictionary:
        if score_csv_path != None and id in score_df['ID_IMAGENOMA'].values:
            df[dts_name][id]['score'] = score_df.loc[score_df['ID_IMAGENOMA'] == id].values[0][score_selection]  # This can be NaN for MedAS
        else:
            # Assign NaN:
            df[dts_name][id]['score'] = np.nan
        
        # Check if there exists a file ending with .nii for each modality:
        for modality in modalities:
            try:
                # Check if there exists a file ending with .nii:
                nifti_files = [f for f in os.listdir(path_dataset + f'/{modality}/{id}') if f.endswith('.nii') or f.endswith('.nii.gz')]
                
                if len(nifti_files) == 0:
                    # If no nifti files, state that the modality is False:
                    df[dts_name][id][modality +'_exists'] = False
                    
                elif len(nifti_files) > 1:
                    # If more than one nifti file exists, save the paths and save all paths: 
                    df[dts_name][id][modality +'_exists'] = True
                    df[dts_name][id]['preprocessed'] = path_dataset + f'/{modality}/{id}/rigid_skull_stripped.nii.gz'
                    df[dts_name][id]['csf'] = path_dataset + f'/{modality}/{id}/csf.nii.gz'
                    df[dts_name][id]['gm'] = path_dataset + f'/{modality}/{id}/gm.nii.gz'
                    df[dts_name][id]['wm'] = path_dataset + f'/{modality}/{id}/wm.nii.gz'
                    df[dts_name][id]['background'] = path_dataset + f'/{modality}/{id}/background.nii.gz'
                    df[dts_name][id]['subc'] = path_dataset + f'/{modality}/{id}/subc.nii.gz'
                    
                    image = sitk.GetArrayFromImage(sitk.ReadImage(df[dts_name][id]['preprocessed']))
                    df[dts_name][id][modality +'_shape'] = image.shape
                    
                else:
                    # If only one nifti file, state that the modality is True and save the path:
                    df[dts_name][id][modality +'_exists'] = True
                    df[dts_name][id][modality +'_path'] = path_dataset + f'/{modality}/{id}/{nifti_files[0]}'
                    
                    # Read the image as an array
                    image = sitk.GetArrayFromImage(sitk.ReadImage(path_dataset + f'/{modality}/{id}/{nifti_files[0]}'))
                    
                    # Save shape of the image:
                    df[dts_name][id][modality +'_shape'] = image.shape
                    
                    """
                    # Save max, min and mean of the image:
                    df[dts_name][id][modality +'_max_min_mean'] = [image.max(), image.min(), image.mean()]
                    """
            except:
                df[dts_name][id][modality +'_exists'] = False
    return df


def save_data_csv(df, path_dataset):
    # Get the name of the dataset from path:
    dts_name = path_dataset.split('/')[-1]
    
    header = list(df[dts_name][list(df[dts_name].keys())[0]].keys())
    my_df = pd.DataFrame(columns = ['dataset', 'ID'] + header)
    
    for id in df[dts_name].keys():
        # Create an empty row and add the dataset and id:
        new_row = {}
        new_row['dataset'] = dts_name
        new_row['ID'] = id

        for key in header:
            if key in df[dts_name][id].keys():
                # Add the key and value to the row:
                new_row[key] = df[dts_name][id][key]
            
        # Append the dictionary to the dataframe:
        my_df = my_df.append(new_row, ignore_index=True)
    
    # Save the dataframe as a csv file with separator ';':
    my_df.to_csv( os.path.dirname(os.path.dirname(path_dataset)) + f'/metadata/{dts_name}.csv', sep=';', index=False)
    return



### Process the chosen datasets:
This will create a /metadata/{dataset_name}.csv file with the for each dataset name in the list "datasets"

In [36]:
# Give the path to the dataset, process and save it:
path_dataset = '../data/datasets/ImaGenoma'

df = process_dataset(path_dataset, score_csv_path = os.path.join(path_dataset,'DIET_Sex_Age.csv'))
save_data_csv(df, path_dataset)

20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 20
20 2

ValueError: cannot set a row with mismatched columns

### Update the metadata.csv
In this case we read all csv files in the metadata folder and concatenate them

In [30]:
metadata_folder = '../data/metadata/'
# Make a list of all csv files in the metadata folder except the metadata.csv file:
csv_files = [f for f in os.listdir(metadata_folder) if f.endswith('.csv') and not f.startswith('.') and not f.startswith('metadata')]

# Concatenate all dataframe in a single dataframe: 
for i, csv_file in enumerate(csv_files):
    if i == 0:
        df = pd.read_csv(f'{metadata_folder}/{csv_file}', sep=';')
    else:
        df = pd.concat([df, pd.read_csv(f'{metadata_folder}/{csv_file}', sep=';')], ignore_index=True)

# Save the dataframe as a csv file:
df.to_csv(f'{metadata_folder}/metadata.csv', sep=';', index=False)

### Import and get to know the "ImageDataset" class

In [32]:
# If import not working, copy paste the class from the metadata.py file:
from metadata import *

# Create an ImageDataset object:
mtd = DatasetMetadata('ImaGenoma','T1_b')

# Print the dataset name and the modality:
print('Name of the dataset: ', mtd.dataset)
print('Name of the modality:', mtd.modality)

# Print the dataframe: (note that the order is set by the ID number and not the index)
mtd.df

Name of the dataset:  ImaGenoma
Name of the modality: T1_b


,dataset,ID,score,T1_exists,T1_path,T1_shape,T2_FLAIR_exists,T2_FLAIR_path,T2_FLAIR_shape,T1_b_exists,preprocessed,csf,gm,wm,background,subc,T1_b_shape,T2_FLAIR_b_exists,T2_FLAIR_b_path,T2_FLAIR_b_shape
593,ImaGenoma,000004-01,NaN,True,../data/datasets/ImaGenoma/T1/000004-01/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000004-01/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000004-01/rigi...,../data/datasets/ImaGenoma/T1_b/000004-01/csf....,../data/datasets/ImaGenoma/T1_b/000004-01/gm.n...,../data/datasets/ImaGenoma/T1_b/000004-01/wm.n...,../data/datasets/ImaGenoma/T1_b/000004-01/back...,../data/datasets/ImaGenoma/T1_b/000004-01/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000004-0...,"(24, 464, 384)"
351,ImaGenoma,000005-00,12.0,True,../data/datasets/ImaGenoma/T1/000005-00/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000005-00/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000005-00/rigi...,../data/datasets/ImaGenoma/T1_b/000005-00/csf....,../data/datasets/ImaGenoma/T1_b/000005-00/gm.n...,../data/datasets/ImaGenoma/T1_b/000005-00/wm.n...,../data/datasets/ImaGenoma/T1_b/000005-00/back...,../data/datasets/ImaGenoma/T1_b/000005-00/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000005-0...,"(24, 464, 384)"
502,ImaGenoma,000007-01,NaN,True,../data/datasets/ImaGenoma/T1/000007-01/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000007-01/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000007-01/rigi...,../data/datasets/ImaGenoma/T1_b/000007-01/csf....,../data/datasets/ImaGenoma/T1_b/000007-01/gm.n...,../data/datasets/ImaGenoma/T1_b/000007-01/wm.n...,../data/datasets/ImaGenoma/T1_b/000007-01/back...,../data/datasets/ImaGenoma/T1_b/000007-01/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000007-0...,"(24, 464, 384)"
183,ImaGenoma,000013-00,8.0,True,../data/datasets/ImaGenoma/T1/000013-00/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000013-00/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000013-00/rigi...,../data/datasets/ImaGenoma/T1_b/000013-00/csf....,../data/datasets/ImaGenoma/T1_b/000013-00/gm.n...,../data/datasets/ImaGenoma/T1_b/000013-00/wm.n...,../data/datasets/ImaGenoma/T1_b/000013-00/back...,../data/datasets/ImaGenoma/T1_b/000013-00/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000013-0...,"(24, 464, 384)"
310,ImaGenoma,000015-01,9.0,True,../data/datasets/ImaGenoma/T1/000015-01/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000015-01/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000015-01/rigi...,../data/datasets/ImaGenoma/T1_b/000015-01/csf....,../data/datasets/ImaGenoma/T1_b/000015-01/gm.n...,../data/datasets/ImaGenoma/T1_b/000015-01/wm.n...,../data/datasets/ImaGenoma/T1_b/000015-01/back...,../data/datasets/ImaGenoma/T1_b/000015-01/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000015-0...,"(24, 464, 384)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,ImaGenoma,805688-01,11.0,True,../data/datasets/ImaGenoma/T1/805688-01/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/805688-01/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/805688-01/rigi...,../data/datasets/ImaGenoma/T1_b/805688-01/csf....,../data/datasets/ImaGenoma/T1_b/805688-01/gm.n...,../data/datasets/ImaGenoma/T1_b/805688-01/wm.n...,../data/datasets/ImaGenoma/T1_b/805688-01/back...,../data/datasets/ImaGenoma/T1_b/805688-01/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/805688-0...,"(24, 464, 384)"
165,ImaGenoma,805688-02,9.0,True,../data/datasets/ImaGenoma/T1/805688-02/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/805688-02/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/805688-02/rigi...,../data/datasets/ImaGenoma/T1_b/805688-02/csf....,../data/datasets/ImaGenom

### Access the data in a "for" loop

In [33]:
# Save the id_dict and the id_list:
id_dict = mtd.id_dict
IDs = mtd.IDs

# print the 10 first preprocessed image paths:
for id in IDs[0:10]:
    print('Current ID: ', id, '. Path to image: ', id_dict[id]['preprocessed'])

Current ID:  000004-01 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000004-01/rigid_skull_stripped.nii.gz
Current ID:  000005-00 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000005-00/rigid_skull_stripped.nii.gz
Current ID:  000007-01 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000007-01/rigid_skull_stripped.nii.gz
Current ID:  000013-00 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000013-00/rigid_skull_stripped.nii.gz
Current ID:  000015-01 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000015-01/rigid_skull_stripped.nii.gz
Current ID:  000018-01 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000018-01/rigid_skull_stripped.nii.gz
Current ID:  000020-01 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000020-01/rigid_skull_stripped.nii.gz
Current ID:  000023-00 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000023-00/rigid_skull_stripped.nii.gz
Current ID:  000026-00 . Path to image:  ../data/datasets/ImaGenoma/T1_b/000026-00/rigid_skull_stripped.nii.gz
C

In [136]:
# Also filter the dataframe by interval:
interval = [[0, 5], [9, 14]]
mtd = DatasetMetadata( 'ImaGenoma', 'T1_b', interval = interval)
mtd.df

,dataset,ID,score,T1_exists,T1_path,T1_shape,T2_FLAIR_exists,T2_FLAIR_path,T2_FLAIR_shape,T1_b_exists,preprocessed,csf,gm,wm,background,subc,T1_b_shape,T2_FLAIR_b_exists,T2_FLAIR_b_path,T2_FLAIR_b_shape
351,ImaGenoma,000005-00,12.0,True,../data/datasets/ImaGenoma/T1/000005-00/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000005-00/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000005-00/rigi...,../data/datasets/ImaGenoma/T1_b/000005-00/csf....,../data/datasets/ImaGenoma/T1_b/000005-00/gm.n...,../data/datasets/ImaGenoma/T1_b/000005-00/wm.n...,../data/datasets/ImaGenoma/T1_b/000005-00/back...,../data/datasets/ImaGenoma/T1_b/000005-00/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000005-0...,"(24, 464, 384)"
310,ImaGenoma,000015-01,9.0,True,../data/datasets/ImaGenoma/T1/000015-01/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000015-01/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000015-01/rigi...,../data/datasets/ImaGenoma/T1_b/000015-01/csf....,../data/datasets/ImaGenoma/T1_b/000015-01/gm.n...,../data/datasets/ImaGenoma/T1_b/000015-01/wm.n...,../data/datasets/ImaGenoma/T1_b/000015-01/back...,../data/datasets/ImaGenoma/T1_b/000015-01/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000015-0...,"(24, 464, 384)"
895,ImaGenoma,000018-01,9.0,True,../data/datasets/ImaGenoma/T1/000018-01/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000018-01/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000018-01/rigi...,../data/datasets/ImaGenoma/T1_b/000018-01/csf....,../data/datasets/ImaGenoma/T1_b/000018-01/gm.n...,../data/datasets/ImaGenoma/T1_b/000018-01/wm.n...,../data/datasets/ImaGenoma/T1_b/000018-01/back...,../data/datasets/ImaGenoma/T1_b/000018-01/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000018-0...,"(24, 464, 384)"
1000,ImaGenoma,000049-00,9.0,True,../data/datasets/ImaGenoma/T1/000049-00/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000049-00/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000049-00/rigi...,../data/datasets/ImaGenoma/T1_b/000049-00/csf....,../data/datasets/ImaGenoma/T1_b/000049-00/gm.n...,../data/datasets/ImaGenoma/T1_b/000049-00/wm.n...,../data/datasets/ImaGenoma/T1_b/000049-00/back...,../data/datasets/ImaGenoma/T1_b/000049-00/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000049-0...,"(24, 464, 384)"
195,ImaGenoma,000057-00,10.0,True,../data/datasets/ImaGenoma/T1/000057-00/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/000057-00/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/000057-00/rigi...,../data/datasets/ImaGenoma/T1_b/000057-00/csf....,../data/datasets/ImaGenoma/T1_b/000057-00/gm.n...,../data/datasets/ImaGenoma/T1_b/000057-00/wm.n...,../data/datasets/ImaGenoma/T1_b/000057-00/back...,../data/datasets/ImaGenoma/T1_b/000057-00/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/000057-0...,"(24, 464, 384)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,ImaGenoma,805670-01,10.0,True,../data/datasets/ImaGenoma/T1/805670-01/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/805670-01/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/805670-01/rigi...,../data/datasets/ImaGenoma/T1_b/805670-01/csf....,../data/datasets/ImaGenoma/T1_b/805670-01/gm.n...,../data/datasets/ImaGenoma/T1_b/805670-01/wm.n...,../data/datasets/ImaGenoma/T1_b/805670-01/back...,../data/datasets/ImaGenoma/T1_b/805670-01/subc...,"(193, 229, 193)",True,../data/datasets/ImaGenoma/T2_FLAIR_b/805670-0...,"(24, 464, 384)"
918,ImaGenoma,805675-01,10.0,True,../data/datasets/ImaGenoma/T1/805675-01/3D_T1W...,"(112, 176, 180)",True,../data/datasets/ImaGenoma/T2_FLAIR/805675-01/...,"(24, 464, 384)",True,../data/datasets/ImaGenoma/T1_b/805675-01/rigi...,../data/datasets/ImaGenoma/T1_b/805675-01/csf....,../data/datasets/ImaGen